In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm
import openexplorer as oe
import numpy as np
from simtk import unit
from simtk.openmm import app

# Successive confinement

S. V. Krivov, S. F. Chekmarev and M. Karplus, Phys. Rev. Lett. 88, 3, 2002.


In [3]:
modeller = msm.convert('alanine_tetrapeptide.pdb', to_form='openmm.Modeller')

topology = modeller.topology
positions = modeller.positions

forcefield = app.ForceField('amber10.xml', 'amber10_obc.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.NoCutoff, #constraints=app.HBonds,
                                implicitSolvent=app.OBC2, soluteDielectric=1.0, solventDielectric=78.5)

In [4]:
explorer = oe.Explorer(topology, system, platform='CUDA')
explorer.set_coordinates(positions)

In [5]:
exploration = oe.exploration_campaign.SuccessiveConfinement(explorer)

In [6]:
exploration.run(1, progress_bar=True)

100%|██████████| 250/250 [00:59<00:00,  4.20it/s]


Podemos calcular la distancia entre configuraciones para hacer un mapa por proximidad: mds o red con threshold.

In [17]:
exploration.ktn.network.edges[0,5]

{'weight': 1}

In [19]:
exploration.trajectory_inherent_structures

{0: [1,
  1,
  1,
  2,
  1,
  1,
  3,
  4,
  1,
  1,
  5,
  6,
  3,
  3,
  3,
  7,
  8,
  3,
  9,
  10,
  11,
  3,
  9,
  9,
  12,
  9,
  1,
  11,
  3,
  3,
  3,
  13,
  6,
  9,
  14,
  15,
  13,
  3,
  16,
  3,
  17,
  1,
  9,
  1,
  9,
  3,
  9,
  18,
  3,
  9,
  3,
  3,
  19,
  3,
  20,
  9,
  11,
  3,
  3,
  21,
  3,
  3,
  3,
  9,
  3,
  19,
  22,
  23,
  9,
  3,
  3,
  24,
  9,
  3,
  9,
  3,
  11,
  11,
  9,
  22,
  3,
  3,
  25,
  9,
  22,
  3,
  9,
  3,
  9,
  17,
  9,
  26,
  27,
  3,
  3,
  9,
  28,
  3,
  29,
  30,
  3,
  9,
  29,
  3,
  7,
  9,
  17,
  9,
  3,
  31,
  9,
  26,
  9,
  19,
  3,
  9,
  9,
  3,
  29,
  9,
  3,
  3,
  31,
  9,
  32,
  3,
  14,
  3,
  3,
  9,
  3,
  3,
  9,
  3,
  3,
  29,
  29,
  9,
  29,
  9,
  3,
  3,
  33,
  3,
  11,
  27,
  2,
  3,
  9,
  33,
  9,
  3,
  9,
  34,
  35,
  3,
  9,
  6,
  3,
  36,
  13,
  37,
  3,
  9,
  3,
  3,
  3,
  38,
  9,
  3,
  3,
  9,
  39,
  26,
  9,
  3,
  13,
  40,
  3,
  9,
  9,
  9,
  3,
  1,
  3,
  9,
  11,
  3,
